<a href="https://colab.research.google.com/github/VesleAnne/Generative_bot/blob/main/Rl_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 2.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gymnasium as gym
from gymnasium.wrappers.record_video import RecordVideo

In [3]:
import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [5]:
import os
import random
import math
from collections import deque, namedtuple
from itertools import count

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.transforms.functional import to_tensor
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Define the DQN neural network model
class DQN(nn.Module):
    def __init__(self, in_channels, n_actions):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, n_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Define the replay memory
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [8]:
# Initialize parameters
class Params:
    env_name_pong = "PongNoFrameskip-v4"
    env_name_space_invaders = "SpaceInvadersNoFrameskip-v4"
    model = "dqn"
    gpu = "cuda"
    lr = 2.5e-4
    epoch = 10001
    batch_size = 32
    eval_cycle = 500
    ddqn = False

args = Params()

In [9]:
# Set hyperparameters
GAMMA = 0.99
EPS_START = 1
EPS_END = 0.05
EPS_DECAY = 500000
WARMUP = 1000

steps_done = 0
eps_threshold = EPS_START

In [10]:
# Epsilon-greedy action selection
def select_action(state):
    global eps_threshold, steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=args.gpu)

In [11]:
# Initialize environment
env_pong = gym.make(args.env_name_pong)
env_space_invaders = gym.make(args.env_name_space_invaders)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [12]:
# Initialize networks
n_action_pong = env_pong.action_space.n
n_action_space_invaders = env_space_invaders.action_space.n

policy_net_pong = DQN(in_channels=4, n_actions=n_action_pong).to(args.gpu)
policy_net_space_invaders = DQN(in_channels=4, n_actions=n_action_space_invaders).to(args.gpu)

target_net_pong = DQN(in_channels=4, n_actions=n_action_pong).to(args.gpu)
target_net_space_invaders = DQN(in_channels=4, n_actions=n_action_space_invaders).to(args.gpu)

target_net_pong.load_state_dict(policy_net_pong.state_dict())
target_net_pong.eval()

target_net_space_invaders.load_state_dict(policy_net_space_invaders.state_dict())
target_net_space_invaders.eval()

DQN(
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=6, bias=True)
)

In [13]:
# Initialize replay memory
memory_pong = ReplayMemory(50000)
memory_space_invaders = ReplayMemory(50000)

In [14]:
# Initialize optimizer
optimizer_pong = optim.AdamW(policy_net_pong.parameters(), lr=args.lr, amsgrad=True)
optimizer_space_invaders = optim.AdamW(policy_net_space_invaders.parameters(), lr=args.lr, amsgrad=True)

In [15]:
!pip install keyboard -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.3 MB/s eta 0:00:00


In [16]:
import keyboard
from itertools import count
import numpy as np
import numpy as np

# Warm-up phase
print("Warming up...")
warmupstep = 0
for epoch in range(args.epoch):
    obs_pong = env_pong.reset()
    obs_pong = np.array(obs_pong[0])  # Extract the first frame
    obs_pong = torch.FloatTensor(obs_pong).to(args.gpu)
    obs_pong = torch.stack((obs_pong, obs_pong, obs_pong, obs_pong)).unsqueeze(0)

    obs_space_invaders, _ = env_space_invaders.reset()
    obs_space_invaders = torch.FloatTensor(obs_space_invaders).to(args.gpu)
    obs_space_invaders = torch.stack((obs_space_invaders, obs_space_invaders, obs_space_invaders, obs_space_invaders)).unsqueeze(0)

    for step in count():
     #   if keyboard.is_pressed('q'):  # Check if 'q' is pressed
     #       print("Training interrupted.")
     #       break

        warmupstep += 1

        action_pong = torch.tensor([[env_pong.action_space.sample()]]).to(args.gpu)
        #next_obs_pong, reward_pong, terminated_pong, truncated_pong, _ = env_pong.step(action_pong.item())
        #done_pong = terminated_pong or truncated_pong

        # Adjust the unpacking to handle environments that return only four values
        next_obs_pong, reward_pong, terminated_pong, _ = env_pong.step(action_pong.item())
        done_pong = terminated_pong


        reward_pong = torch.tensor([reward_pong], device=args.gpu)
        done_pong = torch.tensor([done_pong], device=args.gpu)
        next_obs_pong = torch.FloatTensor(next_obs_pong).to(args.gpu)
        next_obs_pong = torch.stack((next_obs_pong, obs_pong[0][0], obs_pong[0][1], obs_pong[0][2])).unsqueeze(0)


        action_space_invaders = torch.tensor([[env_space_invaders.action_space.sample()]]).to(args.gpu)
        #next_obs_space_invaders, reward_space_invaders, terminated_space_invaders, truncated_space_invaders, _ = env_space_invaders.step(action_space_invaders.item())
        #done_space_invaders = terminated_space_invaders or truncated_space_invaders
        # Adjust the unpacking to handle environments that return only four values
        next_obs_space_invaders, reward_space_invaders, terminated_space_invaders, _ = env_space_invaders.step(action_space_invaders.item())
        done_space_invaders = terminated_space_invaders

        reward_space_invaders = torch.tensor([reward_space_invaders], device=args.gpu)
        done_space_invaders = torch.tensor([done_space_invaders], device=args.gpu)
        next_obs_space_invaders = torch.FloatTensor(next_obs_space_invaders).to(args.gpu)
        next_obs_space_invaders = torch.stack((next_obs_space_invaders, obs_space_invaders[0][0], obs_space_invaders[0][1], obs_space_invaders[0][2])).unsqueeze(0)

        memory_pong.push(obs_pong, action_pong, next_obs_pong, reward_pong, done_pong)
        memory_space_invaders.push(obs_space_invaders, action_space_invaders, next_obs_space_invaders, reward_space_invaders, done_space_invaders)


Warming up...


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'tuple'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:167: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space with exception: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
  logger.warn(f"{pre} is not with

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 3350 has 14.74 GiB memory in use. Of the allocated memory 14.64 GiB is allocated by PyTorch, and 130.00 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
env_space_invaders.reset()